In [1]:
import copy
import csv
import os
import warnings
from argparse import ArgumentParser

import torch
import tqdm
import yaml
from torch.utils import data
# 개별 json 라벨 파일을 이용해 학습 데이터 리스트 생성
import glob
import json
import os
from nets import nn
from utils import util
from utils.dataset import Dataset
from torch.utils import data
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.patches as patches
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:",device)

device: cuda:0


In [2]:

# 파라미터 및 데이터 경로 설정
with open('utils/args.yaml', errors='ignore') as f:
    params = yaml.safe_load(f) 

params['name']={0: 'pd-l1 negative tumor cell',
  1: 'pd-l1 positive tumor cell'}
label_dir = '../../data/IGNITE/annotations/pdl1/individual/'
image_dir = '../../data/IGNITE/images/pdl1/pdl1/'

label_files = sorted(glob.glob(os.path.join(label_dir, '*.json')))
filenames = []
labels = []
for label_file in label_files:
    with open(label_file) as f:
        data1 = json.load(f)
    img_path = os.path.join(image_dir, data1['image']['file_name'])
    if os.path.exists(img_path):
        filenames.append(img_path)
        temp_labels = []
        for i in range(len(data1['annotations'])):
            if data1['annotations'][i]['category_id']==3:
                continue
            else:
                temp_labels.append([data1['annotations'][i]['category_id'],int(data1['annotations'][i]['bbox'][0]),
                         int(data1['annotations'][i]['bbox'][1]),int(data1['annotations'][i]['bbox'][2]),int(data1['annotations'][i]['bbox'][3])])
        labels.append(temp_labels)


In [3]:
class custom_dataset(data.Dataset):
    def __init__(self, filenames, input_size, params, augment, labels=None, image_infos=None):
        self.params = params
        self.mosaic = augment
        self.augment = augment
        self.input_size = input_size
        if labels is not None:
            self.labels = labels
            self.filenames = filenames
            self.n = len(self.filenames)
            self.image_infos = image_infos if image_infos is not None else [None]*len(filenames)
        else:
            loaded = self.load_label(filenames)
            self.labels = list(loaded.values())
            self.filenames = list(loaded.keys())
            self.n = len(self.filenames)
            self.image_infos = [None]*self.n
        self.indices = range(self.n)
        self.albumentations = Albumentations()
    def __len__(self):
        return self.n
    def __getitem__(self, index):
        index = self.indices[index]
        temp_label = copy.deepcopy(self.labels[index])
        
        image,crop_index=self.load_image(index)
        
        crop_x, crop_y = crop_index
        label=[]
        #y,x,h,w, to x_center,y_center,w,h
        for i in range(len(temp_label)):
            x = temp_label[i][2]
            y = temp_label[i][1]
            w = temp_label[i][4]
            h = temp_label[i][3]
            if x >= crop_x+5 and y >= crop_y+5 and x <= crop_x + self.input_size-5 and y <= crop_y + self.input_size-5:
                temp_label[i][1] = (y+h/2 - crop_y)/ self.input_size
                temp_label[i][2] = (x+w/2 - crop_x)/ self.input_size
                temp_label[i][3] = (h) / self.input_size
                temp_label[i][4] = (w) / self.input_size
                label.append(temp_label[i])

        cls=[]
        box=[]
        for i in range(len(label)):
            cls.append(label[i][0])
            box.append(label[i][1:5])
        cls=np.array(cls)-1 # class index 0부터 시작하도록 변경
        box=np.array(box)
        nl = len(box)
        if self.augment:
            nl = len(box)  # update after albumentations

            # Flip up-down
            if random.random() < self.params['flip_ud']:
                image = np.flipud(image).copy()
                if nl:
                    box[:, 1] = 1 - box[:, 1]
            # Flip left-right
            if random.random() < self.params['flip_lr']:
                image = np.fliplr(image).copy()
                if nl:
                    box[:, 0] = 1 - box[:, 0]

        image = image.transpose((2, 0, 1))
        return torch.from_numpy(image),torch.from_numpy(cls), torch.from_numpy(box), torch.zeros(nl)

    def load_image(self, i):
        image = cv2.imread(self.filenames[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR -> RGB 변환
        h, w = image.shape[:2]
        r = self.input_size / min(h, w)
        
        # 이미지가 input_size보다 큰 경우 (r < 1) -> 랜덤 크롭
        if r < 1:
            # 안전하게 크롭 범위 계산
            max_h = max(0, h - self.input_size)
            max_w = max(0, w - self.input_size)
            h1 = random.randint(0, max_h) if max_h > 0 else 0
            w1 = random.randint(0, max_w) if max_w > 0 else 0
            image = image[h1:h1 + self.input_size, w1:w1 + self.input_size]
        else:
            # 이미지가 input_size보다 작거나 같은 경우 (r >= 1) -> 패딩
            h1 = 0
            w1 = 0
            pad_image = np.ones((self.input_size, self.input_size, 3), dtype=np.uint8)*255
            pad_image[:min(h,self.input_size), :min(w,self.input_size), :] = image[:min(h,self.input_size), :min(w,self.input_size), :]
            image = pad_image
        return image, (h1, w1)


    
    
class Albumentations:
    def __init__(self):
        self.transform = None
        try:
            import albumentations

            transforms = [albumentations.Blur(p=0.01),
                          albumentations.CLAHE(p=0.01),
                          albumentations.ToGray(p=0.01),
                          albumentations.MedianBlur(p=0.01)]
            self.transform = albumentations.Compose(transforms,
                                                    albumentations.BboxParams('yolo', ['class_labels']))

        except ImportError:  # package not installed, skip
            pass

    def __call__(self, image, box, cls):
        if self.transform:
            x = self.transform(image=image,
                               bboxes=box,
                               class_labels=cls)
            image = x['image']
            box = np.array(x['bboxes'])
            cls = np.array(x['class_labels'])
        return image, box, cls

split=[0.9, 0.1]
train_dataset=custom_dataset(filenames[:int(len(filenames)*split[0])],512, params, augment=True, labels=labels[:int(len(filenames)*split[0])])
val_dataset = custom_dataset(filenames[int(len(filenames)*split[0]):],512, params, augment=False, labels=labels[int(len(filenames)*split[0]):])

/usr/local/lib/python3.12/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.3'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/usr/local/lib/python3.12/dist-packages/albumentations/core/composition.py:251: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()


In [ ]:
def collate_fn1(batch):
    samples, cls, box, indices = zip(*batch)

    cls = torch.cat(cls, dim=0)
    box = torch.cat(box, dim=0)

    new_indices = list(indices)
    for i in range(len(indices)):
        new_indices[i] += i
    indices = torch.cat(new_indices, dim=0)

    targets = {'cls': cls,
                'box': box,
                'idx': indices}
    return torch.stack(samples, dim=0), targets


# 모델 및 파라미터 준비
model = nn.yolo_v11_m(len(params['names'])).to(device)
optimizer = torch.optim.SGD(util.set_params(model, params['weight_decay']),
                            2e-5, params['momentum'], nesterov=True)
criterion = util.ComputeLoss(model, params)

# 데이터셋 및 데이터로드 (안전한 함수 사용)
batch_size = 8
# 안전하게 데이터로더 생성하는 함수
def create_safe_loader(dataset, batch_size, is_train=True):
    """
    배치 크기에 맞게 데이터셋을 조정하여 안전하게 데이터로더를 생성하는 함수
    """
    dataset_size = len(dataset)
    
    # 배치 크기가 데이터셋 크기보다 큰 경우 배치 크기 조정
    if dataset_size < batch_size:
        print(f"경고: 데이터셋 크기({dataset_size})가 배치 크기({batch_size})보다 작습니다. 배치 크기를 {dataset_size}로 조정합니다.")
        actual_batch_size = max(1, dataset_size)
    else:
        actual_batch_size = batch_size
    
    # 데이터셋이 배치 크기로 나누어 떨어지는지 확인
    if dataset_size % actual_batch_size != 0:
        print(f"참고: 데이터셋 크기({dataset_size})가 배치 크기({actual_batch_size})로 나누어 떨어지지 않습니다.")
        print(f"마지막 배치는 {dataset_size % actual_batch_size}개의 샘플을 포함합니다.")
    
    # 데이터로더 생성
    loader = torch.utils.data.DataLoader(
        dataset, 
        batch_size=actual_batch_size, 
        shuffle=is_train,
        num_workers=4,
        collate_fn=collate_fn1,
        drop_last=(not is_train)  # 훈련 시에는 마지막 배치 유지, 검증 시에는 마지막 배치 제외
    )
    
    return loader, actual_batch_size
# 안전하게 데이터로더 생성
loader, train_batch_size = create_safe_loader(train_dataset, batch_size, is_train=True)
val_loader, val_batch_size = create_safe_loader(val_dataset, 1, is_train=False)

print(f"최종 훈련 배치 크기: {train_batch_size}")
print(f"최종 검증 배치 크기: {val_batch_size}")



참고: 데이터셋 크기(309)가 배치 크기(4)로 나누어 떨어지지 않습니다.
마지막 배치는 1개의 샘플을 포함합니다.
최종 훈련 배치 크기: 4
최종 검증 배치 크기: 1


In [7]:
from utils.valid import compute_validation_metrics, compute_validation_metrics_with_kappa, get_kappa_interpretation
from utils.valid import visualize_ground_truth_and_prediction_separately
from utils.valid import plot_training_progress


# main.py의 train 함수를 참조한 개선된 학습 루프
train_losses = []
val_maps = []
val_precisions = []
val_recalls = []
val_map50s = []
val_kappas = []  # Cohen's Kappa 추가
epochs = 10000

# 체크포인트 저장을 위한 디렉토리 생성
save_dir = '../../model/2class_yolov11/'
os.makedirs(save_dir, exist_ok=True)
#체크포인트 불러오기 
checkpoint_path = os.path.join(save_dir, 'best_model.pt')
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device,weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    
# main.py 스타일의 설정들
best_map = 0
accumulate = max(round(64 / batch_size), 1)  # gradient accumulation steps
amp_scale = torch.amp.GradScaler()  # mixed precision scaler

print(f"Gradient accumulation steps: {accumulate}")

for epoch in range(epochs):
    # 훈련
    model.train()
    
    # main.py 스타일의 평균 손실 추적
    avg_box_loss = util.AverageMeter()
    avg_cls_loss = util.AverageMeter()
    avg_dfl_loss = util.AverageMeter()
    
    train_pbar = tqdm.tqdm(enumerate(loader), total=len(loader), desc=f'Epoch {epoch+1}/{epochs} Training')
    
    optimizer.zero_grad()
    
    for i, (images, targets) in train_pbar:
        step = i + len(loader) * epoch
        images = images.to(device).float() / 255
        
        # Forward pass with mixed precision
        with torch.amp.autocast('cuda'):
            outputs = model(images)
            loss_box, loss_cls, loss_dfl = criterion(outputs, targets)
        
        # 평균 손실 업데이트
        avg_box_loss.update(loss_box.item(), images.size(0))
        avg_cls_loss.update(loss_cls.item(), images.size(0))
        avg_dfl_loss.update(loss_dfl.item(), images.size(0))
        
        # Loss scaling for gradient accumulation
        loss_box *= batch_size  # loss scaled by batch_size
        loss_cls *= batch_size  # loss scaled by batch_size  
        loss_dfl *= batch_size  # loss scaled by batch_size
        
        total_loss = loss_box + loss_cls + loss_dfl
        
        # Backward pass with gradient scaling
        amp_scale.scale(total_loss).backward()
        
        # Gradient accumulation 및 optimization
        if step % accumulate == 0:
            # Gradient clipping 및 optimization
            amp_scale.step(optimizer)
            amp_scale.update()
            optimizer.zero_grad()
        
        # GPU 메모리 동기화
        torch.cuda.synchronize()
        
        # 진행률 표시 업데이트 (main.py 스타일)
        memory = f'{torch.cuda.memory_reserved() / 1E9:.4g}G'
        s = f'Memory: {memory} | Box: {avg_box_loss.avg:.3f} | Cls: {avg_cls_loss.avg:.3f} | DFL: {avg_dfl_loss.avg:.3f}'
        train_pbar.set_description(f'Epoch {epoch+1}/{epochs} | {s}')
    
    # 에폭 평균 손실 계산
    avg_train_loss = avg_box_loss.avg + avg_cls_loss.avg + avg_dfl_loss.avg
    train_losses.append(avg_train_loss)
    
    # 검증 (Cohen's Kappa 포함)
    precision, recall, map50, mean_ap, kappa = compute_validation_metrics_with_kappa(
        model, val_loader, device, params
    )
    val_maps.append(mean_ap)
    val_precisions.append(precision)
    val_recalls.append(recall)
    val_map50s.append(map50)
    val_kappas.append(kappa)
    
    # F1-score 계산
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # 결과 출력 (Cohen's Kappa 포함)
    print(f"\nEpoch {epoch+1}/{epochs} Results:")
    print(f"  Train Loss - Box: {avg_box_loss.avg:.4f}, Cls: {avg_cls_loss.avg:.4f}, DFL: {avg_dfl_loss.avg:.4f}, Total: {avg_train_loss:.4f}")
    print(f"  Validation - Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}")
    print(f"  mAP@0.5: {map50:.4f}, mAP@0.5:0.95: {mean_ap:.4f}")
    print(f"  Cohen's Kappa: {kappa:.4f} ({get_kappa_interpretation(kappa)})")
    print("-" * 80)
    
    # 베스트 모델 저장 (mAP 기준)
    if mean_ap > best_map:
        best_map = mean_ap
        save_checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'amp_scale_state_dict': amp_scale.state_dict(),
            'train_loss': avg_train_loss,
            'box_loss': avg_box_loss.avg,
            'cls_loss': avg_cls_loss.avg,
            'dfl_loss': avg_dfl_loss.avg,
            'map': mean_ap,
            'map50': map50,
            'precision': precision,
            'recall': recall,
            'f1_score': f1_score,
            'kappa': kappa,
            'params': params
        }
        torch.save(save_checkpoint, os.path.join(save_dir, 'best_model.pt'))
        print(f"🎉 새로운 베스트 모델 저장! mAP: {mean_ap:.4f}, Kappa: {kappa:.4f}")
    
    # 최신 모델도 저장 (main.py 스타일)
    last_checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'amp_scale_state_dict': amp_scale.state_dict(),
        'train_loss': avg_train_loss,
        'box_loss': avg_box_loss.avg,
        'cls_loss': avg_cls_loss.avg,
        'dfl_loss': avg_dfl_loss.avg,
        'map': mean_ap,
        'map50': map50,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'kappa': kappa,
        'params': params
    }
    torch.save(last_checkpoint, os.path.join(save_dir, 'last_model.pt'))
    
    # 100 에폭마다 학습 진행 그래프 생성 및 저장
    if (epoch + 1) % 100 == 0:
        try:
            print(f"\n📊 Epoch {epoch+1} - 학습 진행 상황 그래프 생성 중...")
            plot_training_progress(train_losses, val_maps, val_precisions, val_recalls, val_map50s, epoch+1, save_dir)
        except Exception as e:
            print(f"그래프 생성 중 오류: {e}")
    
    # 개선된 검증 이미지 시각화 (매 10 에폭마다) - 실제 라벨과 예측 라벨을 별도 figure로 표시
    if (epoch+1) % 10 == 0:
        try:
            # 여러 샘플에 대해 시각화
            num_samples = 1 # 샘플 수를 1개
            for sample_idx in range(num_samples):
                print(f"\n📊 Epoch {epoch+1} - 검증 샘플 {sample_idx+1}/{num_samples}:")
                print("=" * 60)
                
                # 실제 라벨과 예측 라벨을 별도 figure로 표시
                sample_idx = random.randint(0, len(val_dataset)-1)
                visualize_ground_truth_and_prediction_separately(
                    model, val_dataset, idx=sample_idx, 
                    epoch=epoch+1, save_dir=save_dir
                )
                
        except Exception as e:
            print(f"시각화 중 오류: {e}")

print("🎯 학습 완료!")
print(f"최종 베스트 mAP: {best_map:.4f}")
print(f"모델 저장 위치: {save_dir}")
print(f"베스트 모델: {os.path.join(save_dir, 'best_model.pt')}")
print(f"최신 모델: {os.path.join(save_dir, 'last_model.pt')}")

# 최종 성능 요약
if val_kappas:
    final_kappa = val_kappas[-1]
    final_map = val_maps[-1]
    final_precision = val_precisions[-1]
    final_recall = val_recalls[-1]
    final_f1 = 2 * (final_precision * final_recall) / (final_precision + final_recall) if (final_precision + final_recall) > 0 else 0
    
    print(f"\n📊 최종 성능 요약:")
    print(f"  mAP@0.5:0.95: {final_map:.4f}")
    print(f"  Cohen's Kappa: {final_kappa:.4f} ({get_kappa_interpretation(final_kappa)})")
    print(f"  F1-score: {final_f1:.4f}")
    print(f"  Precision: {final_precision:.4f}")
    print(f"  Recall: {final_recall:.4f}")

Gradient accumulation steps: 16


Epoch 1/10000 | Memory: 3.11G | Box: 3.278 | Cls: 4.559 | DFL: 3.819: 100%|██████████| 78/78 [00:09<00:00,  7.84it/s] 



Epoch 1/10000 Results:
  Train Loss - Box: 3.2781, Cls: 4.5589, DFL: 3.8185, Total: 11.6556
  Validation - Precision: 0.0265, Recall: 0.0772, F1-score: 0.0395
  mAP@0.5: 0.0157, mAP@0.5:0.95: 0.0052
  Cohen's Kappa: 0.1537 (Slight)
--------------------------------------------------------------------------------
🎉 새로운 베스트 모델 저장! mAP: 0.0052, Kappa: 0.1537


Epoch 2/10000 | Memory: 3.11G | Box: 3.166 | Cls: 4.436 | DFL: 3.569: 100%|██████████| 78/78 [00:10<00:00,  7.42it/s]



Epoch 2/10000 Results:
  Train Loss - Box: 3.1657, Cls: 4.4361, DFL: 3.5690, Total: 11.1708
  Validation - Precision: 0.0241, Recall: 0.0954, F1-score: 0.0385
  mAP@0.5: 0.0146, mAP@0.5:0.95: 0.0045
  Cohen's Kappa: 0.0915 (Slight)
--------------------------------------------------------------------------------


...

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

In [ ]:
# Cohen's Kappa 관련 함수들을 valid.py에서 import
from utils.valid import compute_validation_metrics_with_kappa, get_kappa_interpretation, quick_kappa_test

# 현재 모델의 빠른 Cohen's Kappa 테스트
print("🔍 현재 모델의 Cohen's Kappa 빠른 측정...")
current_kappa = quick_kappa_test(model, val_loader, device)